# Die Demokratisierung digitaler Karten

<img src="../img/veit-duotone.png" alt="Portrait Veit Schiele" style="float: left; width:256px; margin-right:32px; border-radius: 50%;"/>

<p>Veit Schiele</p>
<p>
    • Gründer und Geschäftsführer der <a href="https://cusy.io/de">cusy GmbH</a>, Berlin<br>
    • Autor des <a href="https://www.python4data.science/de/latest/">Python for Data Science Tutorials</a><br>
    • Autor des <a href="https://python-basics-tutorial.readthedocs.io/de/latest/">Python-Basics-Tutorial</a><br>
    <img src="demokratisierung-digitaler-karten.svg"
         alt="QR-Code für die URL slides.cusy.io/protomaps/froscon.html"/>
</p>


Der Markt für digitale Kartendienste wird bislang von proprietären Anbietern wie
Mapbox dominiert – mit hohen Lizenzgebühren, eingeschränkter Kontrolle über die
Daten und einem starken Vendor-Lock-in. In diesem Artikel wird Protomaps als
Open‑Source‑Alternative vorgestellt und dessen Potential für verschiedene
Anwendungsfälle beleuchtet.

## Inhalt

1. Was ist Protomaps? Geschichte, Architektur, Datengrundlage und Verarbeitung, Überprüfen der Daten

Zunächst erfolgt eine kurze Einführung in Vector‑Tiles, die Grundidee hinter
Protomaps sowie die wichtigsten Unterschiede zu herkömmlichen Raster‑ oder
proprietären Tile‑Services. Anschließend wird der komplette Tool‑Stack von
Protomaps – von den Open‑Source‑Datensätzen von OpenStreetMap, über die
effiziente Tile‑Erzeugung, bis hin zur Auslieferung via HTTP/2 und dem schlanken
PMTiles-Format – detaillierter beschrieben.

2. Deployment mit S3, Caddy oder nginx

Ein besonderer Fokus liegt auf dem Deployment.

3. Integration mit gängigen Frontend‑Libraries wie MapLibre³, Leaflet⁴ und OpenLayers³⁶

Praktische Beispiele zeigen, wie eigene Tiles generiert werden können, wie
Protomaps selbst gehostet und Datenschutzanforderungen erfüllt werden können.

4. Wofür könnt ihr Protomaps einsetzen … und wofür nicht?

Abschließend werden die wichtigsten Vorteile gegenüber Mapbox diskutiert –
Kostenfreiheit, Datenkontrolle und Offline‑Fähigkeit – sowie ein kurzer Ausblick
auf zukünftige Entwicklungen im Open‑Source‑Kartierungsökosystem gegeben.

5. Zum Weiterlesen

Der Schwerpunkt von Protomaps liegt ausschließlich auf kachelbasierter Kartografie und interaktiver Visualisierung.

Es gibt jedoch auch Erweiterungen für Geokodierung und Routing:

* Geokodierung
  * Nominatim
  * Photon
  * Pelias

* Routing
  * Valhalla
  * OpenTripPlanner

## 1. Was ist Protomaps?

Protomaps¹ ist ein Open‑Source‑Ökosystem zur

Während herkömmliche Kartendienste wie Mapbox⁵ ihre Daten als proprietären
Service vermarkten, liefert Protomaps eine komplette Open-Source-Lösung:

1. Erstellung

Entwickler können Geodaten z.B. aus OpenStreetMap², Natural Earth oder aus GeoJSON o.ä. einlesen

2. Speicherung

Die Speicherung erfolgt im effizienten PMTiles-Format

3. Auslieferung von Vektor‑Kacheln

die resultierenden Kacheln können über einen leichtgewichtigen HTTP/2‑Server ausgeliefert werden.

## 1. Was ist Protomaps?

### 1.A Wie alles begann

Das Protomaps-Projekt wurde von einer Einzelperson Brandon Liu (`@bdon`⁷) ins
Leben gerufen, die auch weiterhin die Hauptverantwortung für die Pflege trägt.

Das Projekt wurde durch den NGI Zero Core Fund finanziert⁸.

### … und wie es weiterging

Heute übernimmt GitHub Sponsors die Kosten für die Infrastruktur, wie z.B.
den Betrieb der API und den Cloud-Speicher.

## 1. Was ist Protomaps?
### 1.B Architektur

Ein Protomaps-Service besteht aus drei Hauptkomponenten:

| Komponente | Zweck |
| :--- | :--- |
| PMTiles erstellen | Einlesen von Geo‑Daten und Generieren der Tiles, z.B. in einer CI/CD‑Pipeline |
| PMTiles‑Store | Persistente Speicherung der `*.pmtiles`-Dateien z.B. in einem S3-kompatiblen Speicherplattform, die HTTP Range Requests und Cross-Origin Resource Sharing (CORS) unterstützt |
| PMTiles‑Server | HTTP/2‑basierter Web‑Server, der Caching und Range‑Requests unterstützt, z.B. Caddy oder nginx |

Hier zeigt sich schön, wie Protomaps die Unix-Philosophie umsetzt

> *„Mache nur eine Sache und mache sie gut“*

Zentral ist das PMTiles-Format, für Speicher und Web-Server werden andere bekannte
Technologien verwendet.

## 1. Was ist Protomaps?
### 1.C Datengrundlage und Verarbeitung

* PMTiles ist ein Einzeldatei-Archivformat für pyramidal gekachelte Daten.
* Die gekachelten Daten können über Z/X/Y-Koordinaten in HTTP Range Requests adressiert werden
* Bei den Daten kann es sich um
  * kartografische Basiskarten
  * Fernerkundungsdaten
  * JPEG-Bilder
  * o.ä. handeln.

* Die Kacheln und Verzeichnisse sind so angeordnet, dass der Aufwand beim
  Verschieben und Zoomen minimiert wird.
* PMTiles ist jedoch ein schreibgeschütztes Format, das es nicht möglich,
  ein Archiv zu aktualisieren, ohne die gesamte Datei neu zu schreiben.

## 1. Was ist Protomaps?
### 1.D Datenquellen

* Basiskarten aus OpenStreetMap⁶- und Natural Earth¹³-Daten

Die bereitgestellten Basiskarten verwenden OpenStreetMap- und Natural
Earth-Daten zur Generierung der `*.pmtiles`-Dateien.

Es enthält jedoch nicht alle Daten und Tags aus OSM, sondern versucht
ein Gleichgewicht zwischen Kachelgröße und Vollständigkeit herzustellen,
damit es als allgemeine Karte verwendet werden kann.

* Eigene Karten mit `github.com/protomaps/basemaps`

Das Repository enthält ein Java-Planetiler-Profil zum Erstellen von PMTiles. So
lassen sich z.B. mit `addr_housenumber` für `kind=address` auch die Hausnummern
übernehmen.

Die Organisation der Ebenen und Tags basiert auf dem Open-Source-Projekt Tilezen
und ist in Basemap Layers dokumentiert.

* Ausschnitte aus diesen Karten mit `pmtiles extract`¹⁸

* `pmtiles convert`¹⁹ konvertiert MBTiles-Archive in PMTiles

* tippecanoe²⁰ kann auch GeoJSON-, FlatGeobuf- und anderen Dateiformaten

* rio-pmtiles²¹ ist ein Plugin für die Rasterio²²-Python-Bibliothek, mit dem
  GeoTIFFs konvertiert werden können

* GDAL ≥ 3.8.0 bietet native Unterstützung für PMTiles²³

## 1. Was ist Protomaps?
### 1.E Überprüfen der Daten

Überprüfen der erstellten Protomaps-Dateien mit `pmtiles verify
{NAME}.pmtiles` 

Header-Daten anzeigen lassen mit

```console
$ pmtiles show NAME.pmtiles  --header-json
```

```json
{
    "tile_compression": "gzip",
    "tile_type": "mvt",
    "minzoom": 0,
    "maxzoom": 7,
    "bounds": [
        -176.684714,
        -14.37374,
        145.830418,
        71.341223
    ],
    "center": [
        -82.96875,
        37.71024,
        7
    ]
}
```

## 1. Was ist Protomaps?
### 1.E Überprüfen der Daten
Meta-Daten anzeigen lassen mit

```console
$ pmtiles show NAME.pmtiles --metadata
```

```json
{"description":"NAME.mbtiles","format":"pbf","generator":"tippecanoe v2.8.1","generator_options":"/Users/veit/workspace/protomaps/tippecanoe/tippecanoe -zg '--projection=EPSG:4326' -o NAME.mbtiles -l zcta NAME.json","maxzoom":"7","minzoom":"0","name":"NAME.mbtiles",… "vector_layers":[{"description":"","fields":{"AFFGEOID10":"String","ALAND10":"Number","AWATER10":"Number","GEOID10":"String","NAME10":"String"},"id":"name","maxzoom":7,"minzoom":0}],"version":"2"}
```

### 1.E Überprüfen der Daten

Wollt ihr euch eure PMTiles-Daten anzeigen lassen, könnt ihr den PMTiles
Viewer²⁵ verwenden:

![pmtiles.io Screenshot](pmtiles.png)

## 2. Deployment

### S3

* PMTiles funktioniert auf jedem S3-kompatiblen Cloud-Speicher, der HTTP Range
  Requests unterstützt.
* Zudem ist Cross-Origin Resource Sharing (CORS) ist erforderlich, wenn euer
  Karten-Frontend auf einer anderen Domain als euer Speicher gehostet wird.
* Mit `pmtiles upload` könnt ihr PMTiles in den S3-kompatiblen Cloud-Speicher
  verschieben:

```console
$ pmtiles upload INPUT.pmtiles REMOTE.pmtiles --bucket=s3://BUCKET_NAME
```

## 2. Deployment

### HTTP2-Server

#### Caddy

```
map.cusy.io {
	### CORS Config ###
    header @map_origins Access-Control-Allow-Origin "https://map.cusy.io"
	header @map_origins Access-Control-Allow-Methods "GET, HEAD"
	header @map_origins Access-Control-Allow-Headers "Range, If-Match"
	header @map_origins Access-Control-Expose-Headers "ETag"

	### Tiles ###
	handle_path /tiles/* {
		root * {{ tiles_dir }}
		file_server
	}
}
```

Ihr könnt jedoch auch euren eigenen HTTP-Server, z. B. mit Caddy²⁷ aufetzen.
Neben der `file_server`-Konfiguration für eure `*.pmtiles` aus einem statischen
Verzeichnis `/tiles/` müsst ihr dann nur noch die CORS-Header setzen.

## 2. Deployment

### HTTP2-Server

#### nginx

```
if ($request_method = "OPTIONS") {
  add_header "Access-Control-Max-Age" 3600;
  add_header "Content-Type" "text/plain charset=UTF-8";
  add_header "Access-Control-Allow-Origin" "*" always;
  add_header "Access-Control-Allow-Headers" "DNT,User-Agent,X-Requested-With,If-Modified-Since,Cache-Control,Content-Type,Range" always;
  add_header "Content-Length" 0;
  return 204;
}
```

## Integration

### MapLibre GL JS

PMTiles können auch vom MapLibre GL JS²⁸-Renderer direkt im Browser gerendert
werden, entweder für thematische Overlay-Tilesets oder Basiskarten-Tilesets.

## Integration

```console
$ npm install pmtiles
```

```javascript
import { Protocol } from "pmtiles";

let protocol = new Protocol();
maplibregl.addProtocol("pmtiles",protocol.tile);
```

Um PMTiles direkt lesen zu können, benötigt ihr die JavaScript-Bibliothek
PMTiles²⁹.

Sie enthält ein Plugin für MapLibre GL, das dessen
`addProtocol`-Funktion³⁰ nutzt.

## Integration

### MapLibre GL JS

Ihr könnt auch eine Karte mit eurem Stil rendern mit dem NPM-Paket
@protomaps/basemaps³⁵.

Das gibt euch eine umfassende Kontrolle bei wenig Code.

```typescript
  getStyle(): StyleSpecification {
    const colorScheme = this.getActiveColorScheme();
    const flavors = this.normalizeFlavors(this.options.flavors);
    const sprites =
      colorScheme == "dark" && flavors.invertDark ? "light" : colorScheme;
    return {
      version: 8,
      glyphs: "https://map.cusy.io/fonts/{fontstack}/{range}.pbf",
      sprite: "https://map.cusy.io/sprites/v4/${sprites}",
      sources: {
        [PMTILES_SOURCE]: {
          type: "vector",
          url: "pmtiles://${this.tilesURL}",
          attribution:
            "Map data: <a href="https://www.openstreetmap.org/copyright" target="_blank">OpenStreetMap</a>",
        },
      },
      layers: layers(PMTILES_SOURCE, flavors[colorScheme], DEFAULT_LAYERS_OPTS),
    };
  }
```

`layers` von `@protomaps/basemaps` ist eine Funktion mit den folgenden drei
Argumenten:

* der Quellenname der Basiskarte
* Ein Flavor-Objekt; die Standardeinstellungen können unter `namedFlavor` mit
`light`, `dark`, `white`, `black`, `grayscale` abgerufen werden.
* Ein `options`-Objekt für Beschriftungen, Sprachschlüssel und Label.

## Integration

### Leaflet⁴

Als Alternative zu MapLibre können PMTiles auch von `Leaflet`⁴ im Browser
gerendert werden.

```console
$ npm install protomaps-leaflet
```

```html
<script src="protomaps-leaflet.js"></script>
<script>
    const map = L.map("map")
    var layer = protomapsL.leafletLayer({url:"FILE.pmtiles", flavor: "light", lang: "en"})
    layer.addTo(map)
</script>
```

## Integration

### OpenLayers³⁶

```console
$ npm install ol-pmtiles
```

```javascript
import "./style.css";
import { Map, View } from "ol";
import VectorTile from "ol/layer/VectorTile";
import { PMTilesVectorSource } from "ol-pmtiles";
import { Style, Stroke, Fill } from "ol/style";
import { useGeographic } from "ol/proj";
```

## Integration

### OpenLayers

```javascript
const vectorLayer = new VectorTile({
  declutter: true,
  source: new PMTilesVectorSource({
    url: "https://r2-public.protomaps.com/protomaps-sample-datasets/nz-buildings-v3.pmtiles",
    attributions: ["© Land Information New Zealand"],
  }),
  style: new Style({
    stroke: new Stroke({
      color: "gray",
      width: 1,
    }),
    fill: new Fill({
      color: "rgba(20,20,20,0.9)",
    }),
  }),
});
```

## Integration

### OpenLayers

```javascript
useGeographic();

const map = new Map({
  target: "map",
  layers: [vectorLayer],
  view: new View({
    center: [172.606201,-43.556510],
    zoom: 7,
  }),
});
```

## 4. Wofür könnt ihr Protomaps einsetzen?

Werfen wir einen Blick auf verschiedene Anwendungen, bei denen sich Protomaps als transformativ erweisen kann.

* Städte und Gemeinden

Städte und Gemeinden können ihr kommerzielles Kartensystem durch eine Protomaps-basierte Lösung ersetzen, um
* Gebietsinformationen
* Infrastrukturprojekte und
* kommunale Ressourcen
anzuzeigen.

Durch die selbst gehostete Implementierung  
* entfallen wiederkehrende Lizenzkosten
* kommt die Möglichkeit hinzu, auf den Karten lokale Orientierungspunkte und Gemeindegrenzen hinzuzufügen, die zuvor mit kommerziellen Diensten nicht oder nur schwer zu markieren waren.

* Offline-Karten

Protomaps ermöglicht die Erstellung von offline-fähigen Kartentools in Gebieten mit sporadischer oder fehlender Internetverbindung.

Durch die Verteilung von PMTiles-Dateien, die detaillierte lokale und regionale Karten enthalten, kann auf interaktive Kartentools zugegriffen werden, ohne dass ein ständiger Internetzugang erforderlich ist.

* Datenschutzkritische Anwendungen

Z.B. können Gesundheitsdienstleister Protomaps nutzen, um Tools zur Standortbestimmung von Einrichtungen zu erstellen.

Dabei können die Standortdaten auf dem Gerät der Endanwender*innen verbleiben.

* Spezialisierte Tools

PMTiles ermöglicht es Unternehmen, hochspezialisierte Karten mit branchenspezifischen Symbolen und Datenvisualisierungen zu erstellen, die von kommerziellen Kartenanbietern nicht angeboten werden können.

Gleichzeitig kann sichergestellt werden, dass die Karten zugänglich sind.

## 4. Wofür könnt ihr Protomaps einsetzen … und wofür nicht?

PMTiles ist für die webbasierte Anzeige von großen, statischen Datensätzen gedacht, d.h.

* sie basieren auf einer Web-Plattform und nicht auf einer lokalen Desktop-Anwendung.

* die Informationen umfassen insgesamt mehr als ein paar Megabyte, also mehr als auf einmal sinnvoll geladen werden kann.

* der Datenbestand ändert sich höchstens täglich oder nie.

## 4. Wofür könnt ihr Protomaps einsetzen … und wofür nicht?

Wenn eure Anwendung nicht über diese drei Features verfügt, gibt es einfachere Alternativen zu PMTiles:

* GeoJSON

Wenn
* ihr eine webbasierte Karte mit statischen Informationen erstellt,
* eure Daten aber klein sind,
solltet ihr sie als eine einzige GeoJSON-Datei bereitstellen.

Dies erspart euch die mühsame Konvertierung eurer Daten in Kacheln.

Ihr könnt die gleichen Kartendesign- und Interaktionstechniken verwenden wie bei gekachelten Daten.

* PostGIS

Wenn ihr eine webbasierte Karte für einen **großen** Datensatz erstellt, der **dynamisch** ist und **häufig aktualisiert** wird, solltet ihr eure Features in einer transaktionalen Datenbank speichern.

Es ist zwar möglich, eine PMTiles-Datei regelmäßig zu aktualisieren, aber dazu muss die Datei bei jedem Speichern neu geladen werden. Dies mag für tägliche Aktualisierungen in Ordnung sein, aber jede höhere Frequenz erfordert eine ineffiziente Datenübertragung.

* GeoParquet

Wenn ihr große, statische Datensätze erforscht, aber **keine Veröffentlichung im Internet** benötiget, könnt ihr das **Kacheln vermeiden** und die Dateien **direkt mit Desktop-Software visualisieren**.

Das Kacheln mit Tools wie tippecanoe erfordert die vorherige Berechnung allgemeiner Übersichtskacheln und ist für den Abruf kleiner, optimierter Datenstücke über das Internet konzipiert. Wenn das Netzwerk nicht der Engpass ist und ihr euren Datensatz lokal habt, ist QGIS eine ausgezeichnete Open-Source-Lösung für die Visualisierung und Kartenerstellung.

GeoParquet ein relativ neues Format, das große Datensätze effizient speichern kann und mit Open-Source-Datentools interoperabel ist.

* Lonboard

* Tools wie Lonboard ermöglichen die Visualisierung von GeoParquet in Jupyter-Notebooks.
* Für lokale Daten sind GeoParquet und Lonboard jedoch eine großartige Lösung für die explorative Datenanalyse.
* Sie ersparen euch die Konvertierung in ein netzwerkoptimiertes Kachelformat.

## 5. Zum Weiterlesen

¹ Protomaps: https://protomaps.com

² OpenStreetMap: https://www.openstreetmap.org

³ MapLibre: https://maplibre.org

⁴ Leaflet: https://leafletjs.com

⁵ Mapbox: https://www.mapbox.com

⁶ OpenStreetMap (OSM): https://www.openstreetmap.org

⁷ NGI0 Core Fund: https://nlnet.nl/core

⁸ NLnet Foundation: https://nlnet.nl/project/Protomaps/

⁹ HTTP Range Requests: https://developer.mozilla.org/en-US/docs/Web/HTTP/Range_requests

¹⁰ Cross-Origin Resource Sharing (CORS): https://developer.mozilla.org/en-US/docs/Web/HTTP/CORS

¹¹ Basiskarten: https://maps.protomaps.com/builds/

¹² OpenStreetMap: https://openstreetmap.org/

## 5. Zum Weiterlesen

¹³ Natural Earth: https://naturalearthdata.com/

¹⁴ basemaps-Repository: https://github.com/protomaps/basemaps

¹⁵ Planetiler: https://github.com/onthegomap/planetiler

¹⁶ Basemap Layers: https://docs.protomaps.com/basemaps/layers

¹⁷ Tilezen: https://tilezen.readthedocs.io/en/latest/layers/

¹⁸ `pmtiles extract`: https://docs.protomaps.com/pmtiles/cli#extract

¹⁹ `pmtiles convert`: https://docs.protomaps.com/pmtiles/cli#convert

²⁰ tippecanoe: https://github.com/felt/tippecanoe

²¹ rio-pmtiles: https://pypi.org/project/rio-pmtiles/

²² Rasterio: https://rasterio.readthedocs.io/en/stable/

²³ GDAL docs: https://gdal.org/en/stable/drivers/vector/pmtiles.html

²⁴ tilemaker: https://github.com/systemed/tilemaker

## 5. Zum Weiterlesen

²⁵ PMTiles Viewer: https://pmtiles.io/

²⁶ RClone: https://rclone.org/

²⁷ Caddy: https://caddyserver.com/

²⁸ MapLibre GL JS: https://maplibre.org/maplibre-gl-js/docs/

²⁹ PMTiles: https://www.npmjs.com/package/pmtiles

³⁰ MapLibre GL `addProtocol`-Funktion: https://maplibre.org/maplibre-gl-js/docs/API/functions/addProtocol/

³¹ MapLibre-Fontstack: https://maplibre.org/maplibre-style-spec/glyphs/

³² MapLibre-Spritesheet-Assets: https://maplibre.org/maplibre-style-spec/sprite/

³³ Basemaps-Assets-Repository: http://github.com/protomaps/basemaps-assets

³⁴ MapLibre-Stile: https://maplibre.org/maplibre-style-spec/

³⁵ @protomaps/basemaps: https://www.npmjs.com/package/@protomaps/basemaps

³⁶ OpenLayers: https://openlayers.org